# Exploratory Analysis

To begin this exploratory analysis, first import libraries and define functions for plotting the data using matplotlib. Depending on the data, not all plots will be made. (Hey, I'm just a simple kerneling bot, not a Kaggle Competitions Grandmaster!)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


In [ ]:
df1 = pd.read_csv('customertest.csv')
df1.head()

In [ ]:
df = pd.read_csv('customertrain.csv')
df.head()

In [ ]:
df.info()

# Prepare the data  


In [ ]:
columns = ["Gender","Ever_Married","Graduated","Profession","Spending_Score"]
for feature in columns:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])

df = df.drop(["Segmentation"], axis=1)        
df = df.drop(["ID"], axis=1)        
df.head()

# Remove rows with no data  

We dont want to work with rows that do not have a Var_1 value, so lets remove them.

In [ ]:
df.dropna(subset=['Var_1'], inplace=True)
df.info()

# Encode Y
Use fit_transform to encode our multinomial categories into numbers we can work with. Later on, we can change back to labels with  
```
df["Var_1"] = yle.inverse_transform(df["Var_1"])
```

We can also print the list of categories as follows:  
```
print(yle.classes_)
```

In [ ]:
yle = LabelEncoder()
df["Var_1"] = yle.fit_transform(df["Var_1"])
df.head()

# Fill in missing features  

An important part of regression is understanding which features are missing. We can choose to ignore all rows with missing values, or fill them in with either mode, median or mode.

- Mode = most common value
- Median = middle value
- Mean = average

Here is a handy function you can call which will fill in the missing features by your desired method. We will choose to fill in values with the average.  
After funning below, you should see 7992 with no null values.

In [ ]:
def fillmissing(df, feature, method):
    if method == "mode":
        df[feature] = df[feature].fillna(df[feature].mode()[0])
        
    elif method == "median":
        df[feature] = df[feature].fillna(df[feature].median())
        
    else:
        df[feature] = df[feature].fillna(df[feature].mean())

In [ ]:
features_missing= df.columns[df.isna().any()]
for feature in features_missing:
    fillmissing(df, feature= feature, method= "mean")
df.info()

# Extract Y column from the dataframe  

Lets extract our Y column into a seperate array and remove it from the dataframe.

In [ ]:
Y = df["Var_1"]
df = df.drop(["Var_1"], axis=1) 

# Plot correlation matrix

If we have a data set with many columns, a good way to quickly check correlations among columns is by visualizing the correlation matrix as a heatmap. Looking at the matrix, you can see 9 columns that have the highest correlation above 0.38. We are only interested in `life expectancy`, so look at the bottom row for your results.   

In [ ]:
plt.figure(figsize = (24,16))
sns.heatmap(pd.concat([df,Y], axis=1).corr(), annot=True, cmap="coolwarm")

Not great features here, but lets see how we go...  
The only ones that look reasonable are Gender, Ever_Married, Agem Graduated and maybe Work_Experience. You usually prefer these to be above 0.3.

# Get X/Y into arrays

Now copy out our X and y columns into matrix's for easier matrix manipulation later.

In [ ]:
X = df.to_numpy()  # np.matrix(df.to_numpy())
y = Y.to_numpy().transpose()  # np.matrix(Y.to_numpy()).transpose()

m,n = X.shape

# Normalise X  

Now, lets normalise X so the values lie between -1 and 1. We do this so we can get all features into a similar range. We use the following equation  
$X_{(i)} = \frac{x_{(i)}-mean(x)}{max(x)-min(x)}$  
  
The goal to perform standardization is to bring down all the features to a common scale without distorting the differences in the range of the values. This process of rescaling the features is so that they have mean as 0 and variance as 1.


In [ ]:
mu = X.mean(0) # 
sigma = X.std(0) # standard deviation: max(x)-min(x)
xn = (X - mu) / sigma

Add a column of ones to X for easier matrix manipulation of our hypothesis and cost function later

In [ ]:
xo = np.hstack((np.ones((m, 1)), xn))

# Setup our neural network

Then, we can setup the sizes of our neural network, first, below is the neural network we want to put together.  
![title](neuralnetwork.png)

Below initialisations, ensure above network is achieved. So, now you are asking "What are reasonable numbers to set these to?"  
  
`Input layer`   = set to the size of the dimensions  
`Hidden layers` = set to input_layer * 2  
`Output layer`  = set to the size of the labels of Y. In our case, this is 7 categories  


In [ ]:
input_layer_size = n                    # Dimension of features
hidden_layer_size = input_layer_size*2  # of units in hidden layer 
output_layer_size = len(yle.classes_)     # number of labels

# Initialise weights  

Now, we can initialise our weights to random small values (remember these are also called theta’s). For gradient descent, its OK to initalise to zero's, but for neural networks, it works out better if we initialise our weights to some random values. Here we develop a handy function to perform the initialization.  

In [ ]:
def initializeWeights(L_in, L_out):
    epsilon_init = 0.12
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
    return W

In [ ]:
initial_Theta1 = initializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = initializeWeights(hidden_layer_size, output_layer_size)
nn_params = np.concatenate((initial_Theta1.flatten(), initial_Theta2.flatten()), axis=None)

# Sigmoid functions  

Since we are doing classification, we will use sigmoid to evaluate our predictions. A sigmoid function is a mathematical function having a characteristic "S"-shaped curve or sigmoid curve. A common example of a sigmoid function is the logistic function shown in the first figure and defined by the formula  

$$sigmoid(z) = g(z) = \frac{1}{1 + e^{-z}}$$
$$g'(z) = \frac{d}{dz}g(z) = g(z)\left(1 - g(z)\right)$$

In checknn.py the following handy functions are created:  
  
- sigmoid is a handy function to compute sigmoid of input parameter Z
- sigmoidGradient computes the gradient of the sigmoid function evaluated at z. This should work regardless if z is a matrix or a vector.

# Regularization  

We will implement regularization as one of the most common problems data science professionals face is to avoid overfitting. Overfitting gives you a situation where your model performed exceptionally well on train data but was not able to predict test data. Neural network are complex and makes them more prone to overfitting. Regularization is a technique which makes slight modifications to the learning algorithm such that the model generalizes better. This in turn improves the model’s performance on the unseen data as well.  

If you have studied the concept of regularization in machine learning, you will have a fair idea that regularization penalizes the coefficients. In deep learning, it actually penalizes the weight matrices of the nodes.  
We implement regularization in nnCostFunction by passing in a lambda which us used to penalise both the gradients and costs that are calculated. 

# Cost function  

We need a function which can implements the neural network cost function for a two layer neural network which performs classification.  
In checknn.py out costfunction will return  

- gradient should be a "unrolled" vector of the partial derivatives of the neural network  
- the final J which is the cost of this weight.   

Our cost function will do the following:  

- Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices for our 2 layer neural network
- Perform forward propagation to calculate (a) and (z)
$${\left(\Theta_{ji}^{(l)}\right)^2} = -\frac{1}{m}trace\left(y^T\log\left(h_\Theta(X)\right) + ({\bf1} - y)^T\log\left({\bf1} - h_\Theta(X)\right)\right) + \frac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}{\left(\Theta_{ji}^{(l)}\right)^2}$$ 
- Calculate the cost of our forward propagation into J and apply regularization 
$$J(\theta) = -\frac{1}{m}\left[\sum_{i=1}^{m}\sum_{k=1}^{K}{y_k^{(i)}\log\left(h_\Theta(x^{(i)})_k\right) + (1 - y_k^{(i)})\log\left(1 - h_\Theta(x^{(i)})_k\right)}\right] + \frac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}$$  
- Perform backward propagation to calculate (s) and apply regularization 
$$\Delta_{ij}^{(l)} = \sum_m{a_j^{(l)}\delta_i^{(l+1)}}$$
$$D^{(l)} = \frac{1}{m}\Delta^{(l)} + \frac{\lambda}{m}\Theta^{(l)}$$

# Checking our forward/backward propagation  

One difficult thing to understand is if our cost function is performing well. A good method to check this is to run a function called checknn.  
Creates a small neural network to check the backpropagation gradients, it will output the analytical gradients produced by your backprop code and the numerical gradients (computed using computeNumericalGradient). These two gradient computations should result in very similar values.  

If you want to delve more into the theory behind this technique, it is tought in Andrew Ng's machine learning course, week 4.  
You do not need to run this every time, just when you have setup your network.  

In [ ]:
from checknn import *

print('Checking Backpropagation... ')
# Weight regularization parameter (we set this to 1 here).
lambda_ = 1
#  Check gradients by running checkNNGradients
checkNNGradients(lambda_)

In [ ]:
from checknn import *

print('Checking Cost Function (w/ Regularization) ... ')
J, g = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, output_layer_size, xn, y, lambda_)

print(f'Cost at parameters (loaded from ex4weights): {J:f} \n(this value should be about 0.383770)')
g

In [ ]:
print('Training Neural Network... ')

#  Change the MaxIter to a larger value to see how more training helps.
options = {'maxiter': 50, 'disp': True}

#  You should also try different values of lambda
lambda_ = 1;

# Create "short hand" for the cost function to be minimized
fun = lambda nn_params: nnCostFunction(nn_params, input_layer_size, hidden_layer_size, output_layer_size, xn, y, lambda_)[0]
jac = lambda nn_params: nnCostFunction(nn_params, input_layer_size, hidden_layer_size, output_layer_size, xn, y, lambda_)[1]

# Now, costFunction is a function that takes in only one argument (the neural network parameters)
from scipy import optimize as opt
res = opt.minimize(fun, nn_params, method='CG', jac=jac, options=options)
nn_params = res.x
cost = res.fun

print(res.message)

In [ ]:
# Obtain Theta1 and Theta2 back from nn_params
Theta1 = nn_params[:hidden_layer_size * (input_layer_size + 1)].reshape((hidden_layer_size, input_layer_size + 1))
Theta2 = nn_params[hidden_layer_size * (input_layer_size + 1):].reshape((output_layer_size, hidden_layer_size + 1))

print(cost)

In [ ]:
pred = predict(Theta1, Theta2, X)

print(f'Training Set Accuracy: {(pred == y).mean() * 100:f}')